In [2]:
import csv
import os
import pandas as pd
import requests
from airflow.models import Variable
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta
from io import StringIO


from airflow import DAG
from helpers import default_args, embulk_run, read_sql_query, connection_db, \
    numero_immatriculation


/home/quentin/anaconda3/lib/python3.9/site-packages/airflow/models/base.py:49 MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)

/tmp/ipykernel_642475/2156468092.py:6 DeprecationWarning: The `airflow.operators.dummy_operator.DummyOperator` class is deprecated. Please use `'airflow.operators.empty.EmptyOperator'`.

/tmp/ipykernel_642475/2156468092.py:7 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

In [3]:
def telechargement_secmar(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
    req = requests.get(url, headers=headers, stream=True)
    data = StringIO(req.text)
    return pd.read_csv(data)

In [ ]:
def sitrep_transform(df, engine):
    df['numero_immatriculation'] = df['numero_immatriculation'].astype(str)
    df = df[df['numero_immatriculation'] != 'nan']
    df = df[df.type_operation.isin(['SAR', 'MAS'])]
    df = df[df.evenement.isin(filtre_sitrep())]
    df = df[df.resultat_flotteur != 'Non assisté, cas de fausse alerte']
    time_format = "%Y-%m-%dT%H:%M:%SZ"

    df['date'] = df['date_heure_reception_alerte'].apply(
        lambda date: datetime.strptime(date.split('+')[0], time_format).date())
    df = df[['date', 'numero_immatriculation', 'operation_id']]
    df = df.drop_duplicates()

    ## Annualisation
    df = annualisation_sitrep(df)
    #return df
    return recuperation_immat(df, engine)

In [ ]:
def recuperation_immat(df, engine):
    navire = pd.read_sql(
        'select id_nav_flotteur, num_immat_francais from navire', engine)
    navire['numero_immatriculation'] = navire['num_immat_francais'].apply(
        lambda x: numero_immatriculation(x, HASH_SECRET))
    df = pd.merge(navire, df, how='inner', on='numero_immatriculation')
    print(df)
    return df[['annee', 'id_nav_flotteur',
               'sitrep']]  # Modification pour obtention date evenement mer


In [13]:
operations = telechargement_secmar(
    url='https://www.data.gouv.fr/fr/datasets/r/fae6bc13-fe4c-4838-b281-b16628b7babe')
flotteurs = telechargement_secmar(
    url='https://www.data.gouv.fr/fr/datasets/r/ae0e17e4-7117-45f0-80c4-b11b38f31c5c')
df_sitreps = pd.merge(flotteurs, operations, how='inner',
                      on='operation_id')

/tmp/ipykernel_642475/2845657508.py:6 DtypeWarning: Columns (1,2,12) have mixed types. Specify dtype option on import or set low_memory=False.

In [14]:
df = sitrep_transform(df_sitreps)

,annee,numero_immatriculation,sitrep,operation_id
0,2009,013046ba3ae9a7fa2b4e4a9bc75f61afe50b7452,1,1
1,2009,01eab75f2ca190e7201eb969adfd6e98f9483443,1,1
2,2009,04746ef042162b156de531fa8a0d0e64a696711e,1,1
3,2009,048981a085a0afe347a4b8f1118dc0fe561a1477,1,1
4,2009,04a6aaf17a5acd954c6be623c5050ecea07ce469,1,1
...,...,...,...,...
45499,2022,ff6f51dab901684d27a1c6093fa15b8b763950f0,1,1
45500,2022,ff956e259e3f29ee66833c6740186568a07d2948,1,1
45501,2023,4071ce2a4b97928e62db60d99c95ea17919a1a78,1,1
45502,2023,a1c8074aeb42480c4f4cfc44ec1b28555aba9683,1,1


In [19]:
df

,annee,numero_immatriculation,sitrep,operation_id
0,2009,013046ba3ae9a7fa2b4e4a9bc75f61afe50b7452,1,1
1,2009,01eab75f2ca190e7201eb969adfd6e98f9483443,1,1
2,2009,04746ef042162b156de531fa8a0d0e64a696711e,1,1
3,2009,048981a085a0afe347a4b8f1118dc0fe561a1477,1,1
4,2009,04a6aaf17a5acd954c6be623c5050ecea07ce469,1,1
...,...,...,...,...
45499,2022,ff6f51dab901684d27a1c6093fa15b8b763950f0,1,1
45500,2022,ff956e259e3f29ee66833c6740186568a07d2948,1,1
45501,2023,4071ce2a4b97928e62db60d99c95ea17919a1a78,1,1
45502,2023,a1c8074aeb42480c4f4cfc44ec1b28555aba9683,1,1
